In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import requests
from datetime import timedelta
plt.close('all')

GitHubUserURLBase = "https://api.github.com/users/"
GitHubPublicEvents = "/events/public"

class GitHubUser:
    """
    Creates and github user object
    """
    def __init__(self, username):
        self.username = username
    def get_github_user(self):
        """
        method get_github_user gets user details
        """
        UserInfo = requests.get(GitHubUserURLBase + self.username)
        UserInfoUsable = UserInfo.json()
        print(UserInfoUsable)
        return(UserInfoUsable)
    def verify_github_user(self):
        """
        method verify_github_user verifies the user exists
        returns a error if user doesn't exist
        """
        UserInfo = self.get_github_user()
        if 'login' in UserInfo:
            return("User_Exists")
        else:
            raise ValueError("Not a valid GitHub user")
    def get_github_user_activity(self):
        """
        returns the public activity for a user
        """
        UserActivityList = []
        PageNumber = 1
        while PageNumber <= 10:
            PageNumberStr = str(PageNumber)
            print("Getting events Page " + PageNumberStr + " of 10")
            UserActivity = requests.get(GitHubUserURLBase + self.username + GitHubPublicEvents + "?page=" + PageNumberStr)
            UserActivityPage = UserActivity.json()
            if len(UserActivityPage) < 30:
                PageNumber += 10
            UserActivityList = UserActivityList + UserActivityPage
            PageNumber += 1
            # print(len(UserActivityList))
        # print(len(UserActivityList))
        return(UserActivityList)

# User = GitHubUser("detaer")
# User.get_github_user_activity()
    
def UserActivityFrame(userevents):
    ids = []
    types = []
    times = []
    for activity in userevents:
        ids.append(activity['id'])
        types.append(activity['type'])
        times.append(activity['created_at'])
    df = pd.DataFrame({'id' : ids,
        'type' : types,
        'created_at' : times,
        'hour_of_day' : types,
        'day_of_week' : types})
    df['created_at'] = pd.to_datetime(df['created_at'])
    df['created_at'] + timedelta(hours = 8)
    df['hour_of_day'] = df.created_at.dt.hour
    df['day_of_week'] = df.created_at.dt.weekday_name
    return(df)

def GraphDayOfWeek(EventList):
    plt.rcdefaults()
    EventsPerWeekDay = EventList['day_of_week'].value_counts().sort_index().plot(kind='bar')
    plt.show()
    
def GraphHoursOfDay(EventList):
    plt.rcdefaults()
    EventsPerWeekDay = EventList['hour_of_day'].value_counts().sort_index().plot(kind='bar')
    plt.show()


def main():
    UserNameToTest = str(input("Enter the GitHub username of the person you want to creep on:"))
    User = GitHubUser(UserNameToTest)
    ##do a try except to throw an error if the user isn't real, if the user is real, grab their public activity
    try:
        TestUser = User.verify_github_user()
        print(TestUser)
        print("Working!")
    except:
        print("User is not a valid github user!")
        main()
    UserEvents = User.get_github_user_activity()
    UserEventsDF = UserActivityFrame(UserEvents)
    MenuItem = int(input(f"Would you like to graph\n 1) What hours {UserName} pushes code \n 2) What Day of the week {UserName} pushes code \n Go on, graph a thing:"))
    if MenuItem == 1:
        print(f"\nWhat hours of the day is {UserName} pushing code?")
        GraphHoursOfDay(UserEventsDF)
    elif MenuItem == 2:
        print(f"Day of the week does {UserName} push code?")
        GraphDayOfWeek(UserEventsDF)
    else:
        print("I am starting to doubt your commitment to Sparkle Motion")
        print("Try again, this time with feeling...")
        main()

if __name__ == '__main__':
    main()


Enter the GitHub username of the person you want to creep on:detaer
{'message': "API rate limit exceeded for 136.24.165.200. (But here's the good news: Authenticated requests get a higher rate limit. Check out the documentation for more details.)", 'documentation_url': 'https://developer.github.com/v3/#rate-limiting'}
User is not a valid github user!
